<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning_basic/blob/master/ML_RE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⏰ML_RE2의 원핫인코딩 여부에 따른 cross_entropy_error함수의 코드 변화 복습

In [ ]:
#원핫인코딩 false
def cross_entropy_error(y,t):
  if y.ndim == 1: 
    t = t.reshape(1,t.size)
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

#원핫인코딩 true
def cross_entropy_error(y,t): #배치를 고려하진 않았지만 로직은 같다
  delta = 1e-7
  return -np.sum(t*np.log(y+delta))

#교차 엔트로피 오차는 원핫인코딩 여부와 관계없이 "정답일때의" 출력으로만 이루어져있다

✅ML6~ML7

**1. np.zeros VS np.zeros_like**

In [11]:
import numpy as np
a = np.zeros([2,3,3]) #인자(변수)의 shape만큼의 array 리턴
print(a,"\n")
print("b의 형태는")
b = np.zeros_like([2,3,3]) #인자(변수)의 size만큼의 array 리턴
print(b)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]] 

b의 형태는
[0 0 0]


**2. 기울기의 의미**

<하강법 기준>

반드시 기울기는 각 지점에서 낮아지는 방향을 가리킨다.

기울기가 0인 부분을 찾으면 최적의 장소를 찾은 것으로 간주하지만,

극솟값이나 안장점의 예로 알 수 있듯 항상 기울기가 0인 부분이 최솟값은 아니다

기울어진 방향이 꼭 최솟값을 가리키는 것은 아니지만 

**기울기가 가리키는 방향으로 가야 함수(loss)의 값을 줄일 수 있다.**


---

이 논리에 맞추어 기울기 정보를 토대로 가중치 매개변수의 값을 업데이트하고, 

(다음 루프에) 이 값을 토대로 loss 값을 줄여나간다


---
가중치 매개변수 업데이트 방식이 SGD, momentum, Adagrad,Adam..이든 관계없이

기본적으로 기울기 정보(loss.backward()를 통해 얻어지는)를 사용한다

---

numerical_gradient(편미분) 또는 gradient(역전파)의 방법으로

기울기 정보를 얻는다.(loss.backward()를 통해)

편미분이든 역전파이든, 전체 학습 알고리즘은

⭕**가중치 매개변수(parameter)에 대한 손실함수의 기울기 값을 토대로 parameter를 최적화 시키는 것이다**

⭕**결국 손실함수(loss)의 기울기를 줄이는 쪽으로 학습이 쭉 이어진다**

⭕**손실함수(loss)는 "예측결과값"과 "정답 레이블"의 비교값의 loss를 구하는 함수이다**

⭕**이렇듯 예측값과 정답값의 오차를 줄이는 쪽으로 이어지는 것이 신경망 학습의 흐름이다**

ex) numerical_gradient(f,w)  --> 이렇게 loss함수와 w 파라미터가 직접적으로 인자로 들어감

ex) gradient(x,t) --> 사실 역전파는 살짝 다름. 

gradient함수 안에서 forward 와 backward 방향으로 흐르게 한 뒤

(backward를 위해서 loss를 구해야함. 그래서 인자로 입력 데이터와 정답 레이블이 들어감)

구해져있는 각 매개변수의 기울기 값(loss함수에 대한) 을 구해준다

---

⚡지도학습의 알고리즘의 흐름

1. 데이터 불러오기 & 전처리 & 배치

2. optimizer(parameter 갱신방법), criterion(loss함수 구하는방법) 설정

3. Net 구현. 또는 불러오기

4. forward 수행

5. criterion로 [forward로 구한 결과값(ouput)과 정답 레이블] 비교. 

6. 만들어진 단 하나의 스칼라 loss 기준의 backward 계산 (기울기 구하기)

7. backward로 구한 기울기로  optimizer를 통한  parameter 업데이트

8. (결과 출력을 위한) loss 값 누적 & 결과 산출

9. 반복 (입력 데이터의 각 배치가 모두 끝날때까지)

10. 반복을 반복 ( epoch. 전체 데이터를 몇 번 반복해서 볼 것인가)

**3. 파라미터의 종류**

▶하이퍼 파라미터 : 사람이 직접 설정

ex) 학습률 같은 매개변수

▶일반 파라미터 : 훈련 데이터와 알고리즘으로 자동 획득

ex) 가중치, 편향 같은 신경망의 매개변수

**학습률은**

0.01이나 0.001같은 값을 많이 씁니다.

너무 크면 큰 값으로 발산하고

너무 작으면 거의 갱신이 되지 않습니다

이러한 하이퍼 파라미터들은 여러 후보 값 중에서 

시험을 통해 가장 잘 학습하는 값을 찾는 과정을 거칩니다

(ML13에서 다룹니다)

**4. numercial_gradient(self,x,t)**

In [3]:
 def numerical_gradient(self, x, t):
    loss_W = lambda W : self.loss(x,t)
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1']) 
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

위 코드에서 보듯 numercial_gradient의 인자에 

입려 데이터와 정답 레이블이 들어감을 볼 수 있다.

기본적으로 numercial_gradient의 편미분을 통한 기울기 구하기 흐름은,

가중치 매개변수에 대한 loss함수의 기울기 값을 구하는 것이다.

즉, loss가 선행되어 구해져야 각 parameter에 대한 loss의 미분값을 구할 수 있다.

그래서 최초의 numercial_gradient에 입력 데이터와 정답 레이블이 들어감을 알 수 있다.

**5. np.random.choice**

numpy.random.choice(a,size=None,replace=True,p=None)

a : 1차원 배열 또는 정수

size : 정수 또는 튜플

replace : 중복 허용 여부

p : 각 데이터가 선택될 확률

In [2]:
import numpy as np
a = np.random.choice(5,3,True)
a
# 0 이상 5미만인 정수 중 3개를 중복 허용하고 출력한다

array([2, 2, 0])

**6. batch**

이제 입력 데이터가 한개씩이 아닌 batch형태로 들어간다. 

이에 맞게 비교를 위한 정답 레이블로 batch형태로 들어간다.

앞의 원핫인코딩의 경우와 마찬가지로, 전체 학습 알고리즘은 변하지 않는다.

다만 내부 함수 ( ex. gradient(x_batch,t_batch))의 구현이 조금 추가될 뿐이다

(ex. if x.ndim == 1 : ...axis=1..)

사실 나중가면 이렇게 입력 데이터가 1개이든 batch 이든 **통일된 코드**가 완성된다

**7. epoch**

똑같은 학습과정을 몇번 반복하는가.

예를 들어 전체 데이터가 100개이고 배치크기가 10이다.

 이럴 경우 배치를 10번 돌려야 전체 데이터 한바퀴를 돌린 것이다. 

 이 한바퀴가 에폭이며, 이 전체 데이터를 10번 반복하여 돌린다면 에폭은 10이다

⏰예제 코드

In [ ]:
import numpy as np
from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)


train_loss_list = []
train_acc_list = [] # !!
test_acc_list = [] # !!

iters_num = 10000 
train_size = x_train.shape[0] 
batch_size = 100 
learning_rate = 0.1 
network = TwoLayerNet(784, 50,10)

iter_per_epoch = max(train_size/batch_size , 1)  

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  
  #grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch) 

  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0: 
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc :  " + str(train_acc) + ", " + str(test_acc))

**위 코드에서(위 코드는 입문용 단순화 코드임)**

gradient를 사용하면 굳이... loss까지 안구하고 역전파를 구현할 수는 있는듯

역전파의 개념대로 하면 출력값까지만 구하기만 하면 

순전파의 반대방향으로 역전파를 흘려 보내기만 하면 되기 때문이다

그래서 위 코드에서는 역전파를 먼저 돌려 parameter마다 기울기를 구한 뒤 loss를 구하는 모습

---
if i % iter_per_epoch == 0:

의 의미는 그냥 "에폭"일때를 의미한다

iter_per_epoch = 한 에폭당 반복수. 